In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

n: the number of points.

In [2]:
n=4

In [3]:
dist=dict()
dist['s']=np.array([0.6,0.4])
dist['x_0']= (1/n)*np.ones(n)
dist['x_1']= np.concatenate(((1/(n-2))*np.ones(n-2),np.zeros(2)),axis=None)
dist['x']=dist['s'][0]*dist['x_0']+dist['s'][1]*dist['x_1']
dist['t_x']= (1/n)*np.ones(n)

In [4]:
v=(dist['x_0']-dist['x_1'])/dist['x']

In [5]:
e=0.01

sum(gamma) == gamma.T \mathbb{1}

sum(gamma.T) == gamma \mathbb{1}

In [7]:
xi=xi_generate([*range(n)],e)

In [106]:
# simplist
gamma=xi
for repeat in range(100):
    new_gamma=np.diag(dist['x']/sum(gamma.T))@gamma
    new2_gamma=new_gamma@np.diag(dist['t_x']/sum(new_gamma))
    gamma=new2_gamma

In [9]:
theta=abs(gamma.T @ v)*(1-0.3)

In [65]:
bbm1=np.matrix(np.ones(n)).T
px=np.matrix(dist['x']).T
ptx=np.matrix(dist['t_x']).T

In [89]:
I=np.where(~(v==0))[0].tolist() # \overline{\supp(X)}
bbm1=np.matrix(np.ones(n)).T
px=np.matrix(dist['x']).T
ptx=np.matrix(dist['t_x']).T
V=np.matrix(v).T
Ve=np.multiply(V,np.exp(V))
theta=np.matrix(abs(gamma.T @ v)*(1-0.3)).T
gamma_dict=dict()
gamma_dict[0]=np.matrix(xi+0.01)
gamma_dict[1]=np.matrix(np.diag((px/(gamma_dict[0] @ bbm1)).A1))@gamma_dict[0]
gamma_dict[2]=gamma_dict[1]@np.matrix(np.diag((ptx/(gamma_dict[1].T @ bbm1)).A1))
# step 3
J=np.where(~((np.matrix(gamma_dict[2]).T @ np.matrix(v).T).A1 >= -theta))[0].tolist()
if len(J)!=0:
    new3_gamma_tmp=gamma_dict[2] @ np.diag(-theta/((np.matrix(gamma_dict[2]).T @ np.matrix(v).T).A1))
    gamma_dict[3]=gamma_dict[2]
    for j in J:
        gamma_dict[3][I,j]=new3_gamma_tmp[I,j]
# step 4
J=np.where(~((np.matrix(gamma_dict[3]).T @ np.matrix(v).T).A1 <= theta))[0].tolist()
if len(J)!=0:
    new4_gamma_tmp=gamma_dict[3] @ np.diag(theta/((np.matrix(gamma_dict[3]).T @ np.matrix(v).T).A1))
    gamma_dict[4]=gamma_dict[3]
    for j in J:
        gamma_dict[4][I,j]=new4_gamma_tmp[I,j]

ValueError: shape mismatch: value array of shape (4,1) could not be broadcast to indexing result of shape (4,)

In [189]:
bbm1=np.matrix(np.ones(n)).T
px=np.matrix(dist['x']).T
ptx=np.matrix(dist['t_x']).T
V=np.matrix(v).T
theta=np.matrix(abs(gamma.T @ v)*(1-0.3)).T
Ve=np.multiply(V,np.exp(V))
gamma_dict[0]=np.matrix(xi+0.01)
gamma_dict[1]=np.matrix(np.diag((px/(gamma_dict[0] @ bbm1)).A1))@gamma_dict[0]
gamma_dict[2]=gamma_dict[1]@np.matrix(np.diag((ptx/(gamma_dict[1].T @ bbm1)).A1))
J=np.where(~(gamma_dict[2].T @ V >= -theta))[0].tolist()
if len(J)!=0:
    new3_gamma_tmp=gamma_dict[2]@np.matrix(np.diag((-theta/(gamma_dict[2].T@Ve)).A1))
    gamma_dict[3]=np.copy(gamma_dict[2])
    for j in J:
        gamma_dict[3][I,j]=np.copy(new3_gamma_tmp[I,j].A1)
J=np.where(~(gamma_dict[3].T @ V <= theta))[0].tolist()
if len(J)!=0:
    new4_gamma_tmp=gamma_dict[3]@np.matrix(np.diag((theta/(gamma_dict[3].T@Ve)).A1))
    gamma_dict[4]=np.copy(gamma_dict[3])
    for j in J:
        gamma_dict[4][I,j]=np.copy(new4_gamma_tmp[I,j].A1)

In [197]:
min(-theta/(gamma_dict[2].T@Ve),1)

matrix([[-0.06923772],
        [-0.07313193],
        [-0.57876238],
        [-0.70036022]])

In [194]:
Ve

matrix([[-0.34967261],
        [-0.34967261],
        [ 8.82415008],
        [ 8.82415008]])

In [170]:
theta

matrix([[0.09423409],
        [0.09423409],
        [0.09423409],
        [0.09423409]])

In [169]:
gamma_dict[2].T @ Ve

matrix([[ 0.09423409],
        [ 0.09423409],
        [-0.09423409],
        [-0.09423409]])

In [167]:
gamma_dict[4]

matrix([[ 4.25116312e-07,  7.31294264e-03, -7.36585910e-02,
         -7.90727575e-02],
        [ 6.37716980e-03,  1.09679515e-06, -5.52448640e-02,
         -7.90714398e-02],
        [ 5.46587226e-03,  4.70102072e-03, -2.36740029e-06,
         -1.69447271e-02],
        [ 5.46596335e-03,  6.26792317e-03, -1.57847730e-02,
         -1.12957317e-06]])

In [164]:
gamma_dict[3][I,1]

matrix([[-7.31294264e-03],
        [-1.09679515e-06],
        [-4.70102072e-03],
        [-6.26792317e-03]])

In [163]:
gamma_dict[3][I,1]=new3_gamma_tmp[I,1].A1

In [145]:
new3_gamma_tmp=gamma_dict[2]@np.matrix(np.diag((-theta/(gamma_dict[2].T@Ve)).A1))

In [138]:
gamma_dict[2].T@Ve

matrix([[1.36102236],
        [1.28854911],
        [0.16281999],
        [0.13455089]])

In [88]:
J=np.where(~((gamma_dict[2].T @ np.matrix(v).T).A1 >= -theta))[0].tolist()
if len(J)!=0:
    new3_gamma_tmp=gamma_dict[2] @ np.diag(-theta/((np.matrix(gamma_dict[2]).T @ np.matrix(v).T).A1))
    gamma_dict[3]=gamma_dict[2]
    for j in J:
        gamma_dict[3][I,j]=new3_gamma_tmp[I,j]

matrix([[0.25],
        [0.25],
        [0.25],
        [0.25]])

In [50]:
gamma_dict[1]

KeyError: 1

In [ ]:
q_dict=dict()
q_dict[1]=gamma_dict[0]/gamma_dict[1]

In [12]:
gamma_dict=dict()
gamma_dict[0]=xi
I=np.where(~(v==0))[0].tolist() # \overline{\supp(X)}
for repeat in range(4):
    new_gamma=np.diag(dist['x']/sum(gamma.T))@gamma
    new2_gamma=new_gamma@np.diag(dist['t_x']/sum(new_gamma))
    # step 3
    J=np.where(~((np.matrix(new2_gamma).T @ np.matrix(v).T).A1 >= -theta))[0].tolist()
    if len(J)!=0:
        new3_gamma_tmp=new2_gamma @ np.diag(-theta/((np.matrix(new2_gamma).T @ np.matrix(v).T).A1))
        new3_gamma=new2_gamma
        for j in J:
            new3_gamma[I,j]=new3_gamma_tmp[I,j]
    # step 4
    J=np.where(~((np.matrix(new3_gamma).T @ np.matrix(v).T).A1 <= theta))[0].tolist()
    if len(J)!=0:
        new4_gamma_tmp=new3_gamma @ np.diag(theta/((np.matrix(new3_gamma).T @ np.matrix(v).T).A1))
        new4_gamma=new3_gamma
        for j in J:
            new4_gamma[I,j]=new4_gamma_tmp[I,j]
    gamma=new4_gamma


In [15]:
theta

array([0.09423409, 0.09423409, 0.09423409, 0.09423409])

In [18]:
dist['x']

array([0.35, 0.35, 0.15, 0.15])

In [19]:
dist['t_x']

array([0.25, 0.25, 0.25, 0.25])

In [16]:
sum(gamma)

array([0.12552459, 0.12552459, 0.20091186, 0.20091186])

In [17]:
sum(gamma.T)

array([0.22850552, 0.22850552, 0.09793094, 0.09793094])

In [14]:
np.matrix(gamma).T @ np.matrix(v).T

matrix([[ 0.09423409],
        [ 0.09423409],
        [-0.09423409],
        [-0.09423409]])

In [6]:
def xi_generate(x_range,e):
    bin=len(x_range)
    C=np.random.random((bin,bin))
    for i in range(bin):
        for j in range(bin):
            C[i,j]=abs(x_range[i]-x_range[j])/e
    return C

In [ ]:

def algorithms(reg,m,n,g,f,C):
    K=np.exp(-C/C.max()/reg)
    interations=100
    trial=10 #check the optimiality every trial. 
    u=np.ones((n,1))
    u_record=np.ones((n,1))
    for tri in range(trial):
        if sum(abs(u_record-u))<10**(-10):
            for i in range(1,interations):
                v=g/np.dot(K.T,u)
                u=f/np.dot(K,v)
            u_record=u
        else:
            break
    return np.dot(np.diag(u.reshape((1,-1))[0]),np.dot(K,np.diag(v.reshape((1,-1))[0])))

def assess(m,n,g,f,C,output):
    print('sum of violation of f:',sum(abs(np.sum(output,1)-f.reshape(n))))
    print('sum of violation of g:',sum(abs(np.sum(output,0)-g.reshape(m))))
    print('total cost:',sum(sum(output*C)))
    print('entropy:',sum(sum(-output*np.log(output+0.1**3))))
    print('============================================')

def plots(x_range,g,f,output):
    fig = plt.figure(figsize=(3,3))
    gs = fig.add_gridspec(2, 2, width_ratios=(4,1), height_ratios=(1,4),left=0.1,right=0.9,bottom=0.1, top=0.9,wspace=0,hspace=0)
    # Create the Axes.
    ax = fig.add_subplot(gs[1, 0])
    ax.pcolormesh(x_range, x_range, output, cmap='Blues')
    ax.set_xlabel(r'$\mathbb{X}$',fontsize=14)
    ax.set_ylabel(r'$\tilde{\mathbb{X}}$',rotation='horizontal',fontsize=14)
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax_histx = fig.add_subplot(gs[0, 0], sharex=ax) 
    ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)
    #ax_histx.set_title(r'$Pr[x]$',rotation='horizontal')
    #ax_histy.set_title(r'$Pr[\tilde{x}]$')
    ax_histx.tick_params(axis="x", labelbottom=False)
    ax_histx.tick_params(axis="y", labelleft=False)
    ax_histy.tick_params(axis="x", labelbottom=False)
    ax_histy.tick_params(axis="y", labelleft=False)
    ax_histx.plot(x_range,g,color='tab:blue')
    ax_histy.plot(f,x_range,color='tab:green') 
    return fig

def OT(reg,x_range,g,f):
    m=len(g)
    n=len(f)
    C=c_generate(x_range)
    output=algorithms(reg,m,n,g.reshape(m,-1),f.reshape(n,-1),C)
    assess(m,n,g,f,C,output)
    plots(x_range,g,f,output)
    return outputa